# Analyse de données d'amplicons 16S avec QIIME 2

Ce notebook a pour but de réaliser une analyse complète de données de séquençage d'amplicons 16S en utilisant l'écosystème QIIME 2. Les étapes incluent le calcul de la diversité alpha, l'analyse de la diversité bêta, la composition taxonomique et une analyse de redondance basée sur la distance (db-RDA).


## 1. Configuration de l'environnement

Cette section installe QIIME 2 et les dépendances nécessaires. Exécutez ces cellules si vous utilisez Google Colab.


In [ ]:
!git clone https://github.com/thomasbio/AtelierSYMBIOSOL.git Atelier1

In [ ]:
%cd Atelier1/Atelier1/

In [ ]:
# Exécutez ce script pour installer QIIME 2
%run setup_qiime2

# Installation des plugins et autres dépendances
!pip install scikit-bio
!pip install seaborn
!qiime plugins install --url https://github.com/qiime2/q2-longitudinal/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-breakaway/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-emperor/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-feature-classifier/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-taxa/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-diversity/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-gneiss/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-composition/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-sample-classifier/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-deicode/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-songbird/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-longitudinal/archive/master.zip


## 2. Importation et description des données

Dans cette section, nous allons charger les fichiers de données et de métadonnées en tant qu'artefacts QIIME 2.


In [ ]:
import qiime2 as q2
from qiime2.plugins import demux, metadata, feature_table, diversity, emperor

# Charger les métadonnées
metadata_path = 'metadata.tsv'
metadata = q2.Metadata.load(metadata_path)

# Charger la table de contingence
table_path = 'table-emp.qza'
table = q2.Artifact.load(table_path)

# Charger les séquences représentatives
rep_seqs_path = 'rep-seqs.qza'
rep_seqs = q2.Artifact.load(rep_seqs_path)

# Charger la taxonomie
taxo_path = 'taxo-silva128.qza'
taxonomy = q2.Artifact.load(taxo_path)

# Afficher un résumé de la table de contingence
!qiime feature-table summarize --i-table {table_path} --o-visualization table.qzv --m-sample-metadata-file {metadata_path}
!qiime metadata tabulate --m-input-file table.qzv --o-visualization table.qzv


## 3. Analyse de la diversité Alpha (indice de Shannon)

Nous allons maintenant calculer l'indice de diversité de Shannon pour chaque échantillon.


In [ ]:
# Calcul de la diversité alpha (Shannon)
!qiime diversity alpha --i-table {table_path} --p-metric shannon --o-alpha-diversity shannon.qza

# Visualisation des résultats
!qiime diversity alpha-group-significance --i-alpha-diversity shannon.qza --m-metadata-file {metadata_path} --o-visualization shannon_significance.qzv


## 4. Analyse de la diversité Bêta (PCoA)

Cette section est dédiée à l'analyse de la diversité bêta via une Analyse en Coordonnées Principales (PCoA) sur une matrice de distance de Bray-Curtis.


In [ ]:
# Calcul de la matrice de distance (Bray-Curtis)
!qiime diversity beta --i-table {table_path} --p-metric braycurtis --o-distance-matrix bray_curtis_distance_matrix.qza

# Analyse PCoA
!qiime diversity pcoa --i-distance-matrix bray_curtis_distance_matrix.qza --o-pcoa pcoa_results.qza

# Visualisation avec Emperor
!qiime emperor plot --i-pcoa pcoa_results.qza --m-metadata-file {metadata_path} --o-visualization pcoa_emperor.qzv


## 5. Composition Taxonomique

Visualisation de la composition taxonomique des échantillons.


In [ ]:
# Création du barplot taxonomique
!qiime taxa barplot --i-table {table_path} --i-taxonomy {taxo_path} --m-metadata-file {metadata_path} --o-visualization taxa_barplot.qzv


### Heatmap de la composition taxonomique (par classe)


In [ ]:
# Agréger la table au niveau de la classe
!qiime taxa collapse --i-table {table_path} --i-taxonomy {taxo_path} --p-level 3 --o-collapsed-table table-l3.qza

# Créer le heatmap
!qiime feature-table heatmap --i-table table-l3.qza --m-metadata-file {metadata_path} --m-metadata-column body-site --p-normalize --o-visualization heatmap-l3.qzv


## 6. Analyse de Redondance basée sur la Distance (db-RDA) avec Python

Cette analyse permet d'évaluer la relation entre la composition des communautés microbiennes et les variables environnementales numériques. Nous allons exporter la matrice de distance générée par QIIME 2 et utiliser les packages `scikit-bio`, `pandas` et `seaborn` pour réaliser et visualiser l'analyse.


In [ ]:
import pandas as pd
import skbio
from skbio.stats.ordination import rda, pcoa
import seaborn as sns
import matplotlib.pyplot as plt
from qiime2 import Artifact

# Charger la matrice de distance Bray-Curtis et la visualiser en tant qu'objet skbio.DistanceMatrix
bray_curtis_dm = Artifact.load('bray_curtis_distance_matrix.qza').view(skbio.DistanceMatrix)

# Charger les métadonnées en tant que DataFrame pandas
metadata_df = pd.read_csv('metadata.tsv', sep='\t', index_col=0)

# Aligner la matrice de distance et les métadonnées sur les mêmes échantillons
sample_ids = bray_curtis_dm.ids
metadata_df = metadata_df.loc[sample_ids]

# Sélectionner les variables spécifiques pour l'analyse
variables_for_rda = ['pH', 'Sand', 'Carbon', 'AMWD']
# Vérifier si les colonnes existent avant de les sélectionner
existing_vars = [var for var in variables_for_rda if var in metadata_df.columns]

if not existing_vars:
    print("Avertissement : Aucune des colonnes spécifiées (pH, Sand, Carbon, AMWD) n'a été trouvée dans les métadonnées.")
else:
    if len(existing_vars) != len(variables_for_rda):
        missing_vars = set(variables_for_rda) - set(existing_vars)
        print(f"Avertissement : Les colonnes suivantes sont manquantes dans les métadonnées et ne seront pas utilisées : {list(missing_vars)}")
    
    env_vars = metadata_df[existing_vars]

    # 1. Effectuer une PCoA sur la matrice de distance
    pcoa_results = pcoa(bray_curtis_dm)
    
    # 2. Effectuer la RDA sur les résultats de la PCoA avec les variables sélectionnées
    db_rda_results = rda(y=pcoa_results.samples, x=env_vars, scale_y=False)
    
    # Créer un DataFrame pour le plotting des échantillons
    plot_df = db_rda_results.samples.iloc[:, :2]
    plot_df.columns = ['PC1', 'PC2']
    
    # Ajouter les informations de métadonnées pour la coloration
    # Remplacez 'body-site' par la colonne que vous souhaitez utiliser pour la couleur
    plot_df = plot_df.join(metadata_df['body-site'])

    # 3. Visualiser les résultats (échantillons)
    plt.figure(figsize=(12, 10))
    sns.scatterplot(data=plot_df, x='PC1', y='PC2', hue='body-site')
    
    # 4. Ajouter les flèches des variables environnementales (vecteurs)
    arrow_data = db_rda_results.features.iloc[:, :2]
    arrow_data.columns = ['PC1', 'PC2']
    
    # Facteur d'échelle pour une meilleure lisibilité des flèches
    arrow_scale = 2.0 
    
    for var_name, row in arrow_data.iterrows():
        plt.arrow(0, 0, row['PC1'] * arrow_scale, row['PC2'] * arrow_scale, 
                  head_width=0.03, head_length=0.05, fc='red', ec='red', lw=1.5)
        plt.text(row['PC1'] * arrow_scale * 1.15, row['PC2'] * arrow_scale * 1.15, 
                 var_name, color='red', ha='center', va='center', fontsize=12)

    plt.title('db-RDA des communautés microbiennes')
    plt.xlabel(f'PC1 ({db_rda_results.proportion_explained[0]:.2%})')
    plt.ylabel(f'PC2 ({db_rda_results.proportion_explained[1]:.2%})')
    plt.legend(title='body-site')
    plt.grid(True)
    plt.axhline(0, color='grey', linestyle='--')
    plt.axvline(0, color='grey', linestyle='--')
    plt.show()
